In [1]:
import numpy as np
import pandas as pd

In [2]:
# 처음에는 전체 hitsory data에 대해 작성했던 코드인데, df_train_week로 적용하려다보니 이슈 발생
# 추후 코드 조금 더 만져둘 예정
# 사실 mp추천이 당장 중요한 건 아니라 모델부터 건드려 볼 계획

In [3]:
test_answer_week = pd.read_parquet("../input/lg-train-test/test_answer_week.parquet")
test_answer_month = pd.read_parquet("../input/lg-train-test/test_answer_month.parquet")

df_train_week = pd.read_parquet("../input/lg-train-test/train_week.parquet")
df_train_month = pd.read_parquet("../input/lg-train-test/train_month.parquet")

sample_sumbission_week = pd.read_parquet("../input/lg-train-test/sample_sumbission_week.parquet")
sample_sumbission_month = pd.read_parquet("../input/lg-train-test/sample_sumbission_month.parquet")

In [4]:
df_train_week

,profile_id,ss_id,act_target_dtl,album_id,payment,continuous_play,short_trailer,log_dt,log_date
0,3,20220301115653,MKID003,15,NaN,Y,N,2022-03-01 11:57:00,2022-03-01
1,3,20220301115653,MKID003,16,NaN,Y,N,2022-03-01 11:58:00,2022-03-01
2,3,20220301115653,MKID003,17,NaN,Y,N,2022-03-01 11:59:00,2022-03-01
3,3,20220301115653,MKID003,18,NaN,Y,N,2022-03-01 12:01:00,2022-03-01
4,3,20220301115653,MKID003,19,NaN,Y,N,2022-03-01 12:02:00,2022-03-01
...,...,...,...,...,...,...,...,...,...
1005465,33015,20220423100901,MKID003,2276,NaN,N,N,2022-04-23 10:18:00,2022-04-23
1005474,33016,20220423100063,MKID003,491,NaN,N,N,2022-04-23 10:01:00,2022-04-23
1005475,33016,20220423100063,MKID003,492,NaN,Y,N,2022-04-23 10:09:00,2022-04-23
1005476,33019,20220423222605,MKID003,124,NaN,Y,N,2022-04-23 22:26:00,2022-04-23


# 각 유저의 영상시청 마지막 주 찾기
split 하기 전의 전체 데이터에 대해서 효과가 있는 방법이긴 하지만, 목적은 아래와 같다.
* 특정 유저가 마지막으로 행동한 날로부터 일주일 전의 기록에 대해서만 mp를 산출한다.
* 모든 유저의 마지막 행동일자가 상이할 것이기 때문이다.

In [5]:
tmp = df_train_week.groupby('profile_id').log_date.max().reset_index() #해당 고객의 마지막 주를 찾아내서 가장 최근인 순(t_dat.max())으로 인덱스 정렬
tmp.columns = ['profile_id','max_dat'] # 인덱스 네임 다시 배정
tmp.rename(columns = {'log_date':'max_dat'},inplace=True)
df_train_week = df_train_week.merge(tmp,on=['profile_id'],how='left') # 만들어낸 날짜 col로 추가 
df_train_week['diff_dat'] = (df_train_week.max_dat - df_train_week.log_date).dt.days #새 컬럼을 만드는데, 최근거래날짜에서 거래날짜를 뺀 일수.(해당 거래가 그 사람의 마지막 거래로부터 며칠전의 거래인지 확인)
df_train_week = df_train_week.loc[df_train_week['diff_dat']<=6] # 그 사람의 마지막 거래일 기준으로, 그날로부터 6일 전까지의 기록만 train으로 남김
print('Train shape:',df_train_week.shape)

Train shape: (179864, 11)


# 이전에 자주 시청한 영상 목록 확인
* 유저가 이전에 가장 자주 소비한 영상을 찾는다. 
* ct 를 기준으로 정렬한 후 log_date을 사용하면 이전에 자주 시청한 영상을 먼저 추천하고,
* 그 다음으로 resent하게 시청한 영상을 추천할 수 있다. (순서를 매길 수 있다) 

In [6]:
tmp = df_train_week.groupby(['profile_id','album_id'])['log_date'].agg('count').reset_index() 
#유저id와 아이템id별로 특정 날에 이것을 구매한 개수를 일자별로 세고, 싹 집계해 프레임 정렬한다. 
tmp.columns = ['profile_id','album_id','ct'] 
df_train_week = df_train_week.merge(tmp,on=['profile_id','album_id'],how='left')  
df_train_week = df_train_week.sort_values(['ct','log_date'],ascending=False) # 'ct','t_dat' 값 기준으로 내림차순 정렬. 
df_train_week = df_train_week.drop_duplicates(['profile_id','album_id']) 
df_train_week = df_train_week.sort_values(['ct','log_date'],ascending=False) # 혹시 모르니 다시 한 번 정렬해줌
df_train_week.head()

,profile_id,ss_id,act_target_dtl,album_id,payment,continuous_play,short_trailer,log_dt,log_date,max_dat,diff_dat,ct
87315,12202,20220423193371,MKID003,319,NaN,Y,Y,2022-04-23 19:45:00,2022-04-23,2022-04-23,0,420
21713,2794,20220421234885,MKID003,3502,NaN,Y,N,2022-04-22 00:03:00,2022-04-22,2022-04-23,1,417
21714,2794,20220421234885,MKID003,228,NaN,Y,N,2022-04-22 00:04:00,2022-04-22,2022-04-23,1,412
21715,2794,20220421234885,MKID003,1047,NaN,Y,N,2022-04-22 00:06:00,2022-04-22,2022-04-23,1,405
21712,2794,20220421234885,MKID003,2641,NaN,Y,N,2022-04-22 00:01:00,2022-04-22,2022-04-23,1,402


# 연계 시청 영상 목록 만들기 
* 한 영상과 함께 많이 소비된 영상들을 찾는다.
* 연속 시청 비율이 매우 높았으므로 같이 보는 영상들의 패턴을 찾아보고자 한다.
* [한 유저가 소비한 특정 영상 : 해당 영상과 같이 시청된 영상 4개] 추출

In [7]:
# 1. 영상 3개 이하 본 유저 리스트 / len(cold_user) : 670
user_list = df_train_week.profile_id.value_counts().reset_index()
user_list.columns = ['profile_id','view_count']
cold_user = user_list[user_list['view_count']<3].index.values.tolist()

In [8]:
#판매된 아이템 개수를, 아이템 ID별로 개수 집계
vc_ = df_train_week.album_id.value_counts().reset_index()
vc = vc_[vc_.album_id > 3].set_index('index')
pairs = {}
for j,i in enumerate(vc.index.values):
    USERS_ = df_train_week.loc[df_train_week.album_id==i.item(),'profile_id'].unique().tolist()
    USERS = [x for x in USERS_ if x not in cold_user]
    vc2 = df_train_week.loc[(df_train_week.profile_id.isin(USERS))&(df_train_week.album_id!=i.item()),'album_id'].value_counts()
    pairs[i.item()] = [vc2.index[0],vc2.index[1],vc2.index[2],vc2.index[3]]
    
    # pair에 itemID를 key로,같이 구입되고 많이 구입된 item top4을 저장.

In [9]:
pairs 

{16: [15, 17, 19, 38],
 241: [347, 1880, 329, 383],
 339: [190, 124, 52, 125],
 15: [16, 17, 18, 19],
 124: [65, 125, 190, 339],
 38: [16, 15, 39, 225],
 347: [241, 1880, 329, 339],
 125: [65, 124, 339, 190],
 1880: [241, 347, 329, 339],
 190: [339, 124, 125, 65],
 264: [987, 2934, 1301, 16],
 19: [16, 15, 17, 18],
 65: [125, 124, 339, 190],
 17: [16, 15, 18, 19],
 18: [16, 15, 17, 19],
 52: [338, 339, 423, 190],
 329: [241, 1880, 347, 2348],
 224: [16, 38, 15, 17],
 39: [38, 16, 225, 136],
 225: [38, 39, 16, 136],
 423: [1402, 52, 339, 338],
 1402: [423, 339, 1438, 52],
 36: [35, 34, 16, 38],
 416: [417, 419, 418, 16],
 136: [38, 39, 225, 230],
 338: [52, 339, 423, 190],
 383: [241, 329, 347, 1880],
 127: [125, 124, 65, 2029],
 2044: [2031, 2062, 2032, 492],
 230: [136, 38, 39, 225],
 188: [1425, 1426, 1447, 16],
 505: [2491, 2045, 2492, 478],
 183: [612, 999, 996, 16],
 56: [2200, 1583, 2201, 2202],
 478: [2492, 471, 2491, 505],
 229: [16, 224, 38, 15],
 2062: [2044, 492, 491, 2031],

In [10]:
df_train_week['album_id2'] = df_train_week.album_id.map(pairs)

In [ ]:
# RECOMMENDATION OF PAIRED ITEMS
train2 = df_train_week[['profile_id','album_id2']].copy()  
train2 = train2.loc[train2.album_id2.notnull()]   
train2 = train2.drop_duplicates(['profile_id','album_id2'])  
train2 = train2.rename({'album_id2':'album_id'},axis=1)  

In [ ]:
# CONCATENATE PAIRED ITEM RECOMMENDATION AFTER PREVIOUS PURCHASED RECOMMENDATIONS  
df_train_week = df_train_week[['profile_id','album_id']]  
df_train_week = pd.concat([df_train_week,train2],axis=0,ignore_index=True) #유저id와 아이템id 기준으로 상하 concat. 같이 팔린것들을 개별 구매기록처럼 만들어버림
df_train_week.album_id = df_train_week.album_id.astype('int64') 
df_train_week = df_train_week.drop_duplicates(['profile_id','album_id'])  

In [ ]:
import pickle

with open('pairs.pkl', 'wb') as f:
    pickle.dump(pairs, f, protocol=pickle.HIGHEST_PROTOCOL)

# last week mp 추천
* 이전 시청에 이어 함께 시청한 아이템을 추천한 후, mp 25개를 추천한다. 
* 따라서 25개 영상 개수보다 부족한 경우 mp로 채운다.

In [ ]:
train = cudf.read_parquet('train.pqt')  
train.t_dat = cudf.to_datetime(train.t_dat)   
train = train.loc[train.t_dat >= cudf.to_datetime('2020-09-16')] # 마지막 7일만 train에 남겨둠
top12 = ' 0' + ' 0'.join(train.article_id.value_counts().to_pandas().index.astype('str')[:25]) 
print("Last week's top 12 popular items:")
print( top25 )

# submission

In [ ]:
# 코드 보충 계획